In [10]:
import os
import os.path as osp
import cv2
import shutil
import random
import pandas as pd
from tqdm.notebook import tqdm
from augmentations import augmentation_choices, transform_image

In [6]:
def make_subdir(subdir):
    if not osp.isdir(subdir):
        try:
            os.mkdir(subdir)
        except FileNotFoundError:
            print("Make sure the parent folder exists")

In [33]:
split_file = 'kitti_eigen_test.txt'
input_prefix_path = "E:\Thesis_Datasets\\kitti_full\\"
input_depth_folder = 'val'
valid_image_count = 0
output_prefix_path = "E:\Thesis_Datasets\\Augmented_Images\\"
output_image_folder = "Data\\"
output_depth_folder = "Depth\\"
image_data = []

with open(split_file) as f:
    for line in tqdm(f, desc="Augmenting input files...", total=697):
        img_name = line.strip().split(" ")[0]
        img_name = osp.join(input_prefix_path, img_name)
        depth_file = line.strip().split(" ")[1]
        depth_map = osp.join(input_prefix_path, input_depth_folder, depth_file)

        if osp.isfile(img_name) and osp.isfile(depth_map):
            valid_image_count += 1

            # Ensure the folder exists
            make_subdir(osp.join(output_prefix_path, output_depth_folder))

            #   Copy depth map
            shutil.copy(depth_map,
                        osp.join(output_prefix_path, output_depth_folder, depth_map.split('/')[-1]))

            #   For each augmentation type:
            for aug in augmentation_choices:
                make_subdir(osp.join(output_prefix_path, output_image_folder, aug))

                # if aug = affine, depth map needs to be augmented with the same scale, rotation and shear
                if aug == 'affine':
                    make_subdir(osp.join(output_prefix_path, output_depth_folder, aug))

                    scale=(1.1, 1.1)
                    rotation=random.uniform(-0.05, 0.05)
                    shear=random.uniform(-0.1, 0.1)

                    img = transform_image(aug, cv2.imread(img_name), scale, rotation, shear)
                    augmented_depth = transform_image(aug, cv2.imread(depth_map), scale, rotation, shear)

                    # Save transformed depth map in Depth/affine/xxxxxxx_affine.png
                    cv2.imwrite(osp.join(output_prefix_path, 
                                     output_depth_folder, 
                                     aug, 
                                     img_name[img_name.rfind('/')+1:-4] + "_affine.png"),
                                augmented_depth)
                    
                    # Insert [rgb_file, depth_file, augmentation_type] into list
                    image_data.append([osp.join(output_image_folder, aug, img_name.split('/')[-1]),
                                       osp.join(output_depth_folder, aug, depth_file.split('/')[-1][:-4] + "_affine.png"),
                                       aug])
                                       
                else:
                    img = transform_image(aug, cv2.imread(img_name))
                    # Insert [rgb_file, depth_file, augmentation_type] into list
                    image_data.append([osp.join(output_image_folder, aug, img_name.split('/')[-1]),
                                       osp.join(output_depth_folder, depth_file.split('/')[-1]),
                                       aug])
                    
                # Save file with new name in /<augmentation_type>/ folder
                cv2.imwrite(osp.join(output_prefix_path, 
                                     output_image_folder, 
                                     aug, 
                                     img_name.split('/')[-1]),
                            img)
            if valid_image_count > 3:
                break

print(f"Total: {valid_image_count} input files")


Augmenting input files...:   0%|          | 0/697 [00:00<?, ?it/s]

Total: 4 input files


In [34]:
image_data_df = pd.DataFrame(image_data, columns=['image', 'depth', 'augmentation'])
image_data_df.head(50)

,image,depth,augmentation
0,Data\gaussian\0000000069.png,Depth\0000000069.png,gaussian
1,Data\localvar\0000000069.png,Depth\0000000069.png,localvar
2,Data\poisson\0000000069.png,Depth\0000000069.png,poisson
3,Data\salt\0000000069.png,Depth\0000000069.png,salt
4,Data\pepper\0000000069.png,Depth\0000000069.png,pepper
5,Data\s&p\0000000069.png,Depth\0000000069.png,s&p
6,Data\speckle\0000000069.png,Depth\0000000069.png,speckle
7,Data\affine\0000000069.png,Depth\affine\0000000069_affine.png,affine
8,Data\rain\0000000069.png,Depth\0000000069.png,rain
9,Data\shadow\0000000069.png,Depth\0000000069.png,shadow


Create a split_file_<aug>.txt for each augmentation type.

In [30]:
for aug in augmentation_choices[1:]:
    subset = image_data_df.loc[image_data_df.augmentation == aug]
    subset[['image', 'depth']].to_csv('split_file_'+aug+'.txt', sep=' ', index=False, header=False)
image_data_df.to_pickle('image_data_df.pkl')

In [19]:
# valid_image_count is 161. Need to figure out why.
valid_image_count

161